In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import dot, stack, concatenate
import pandas as pd

In [18]:
np.version.version

'1.22.3'

In [2]:
time=np.array([24,32,48,56,np.nan,70,72,75,80,96])
temp=np.array([102.8,104.5,106.5,107.0,107.1,105.1,103.9,np.nan,103.2,102.1])

In [4]:
temp = temp.copy()
temp = np.nan_to_num(temp, nan=-1)
temp = np.ma.masked_values(temp, value=-1)

time = time.copy()
time = np.nan_to_num(time, nan=-1)
time = np.ma.masked_values(time, value=-1)
time.shape, temp.shape
n, p = 10, 2

In [6]:
time

masked_array(data=[24.0, 32.0, 48.0, 56.0, --, 70.0, 72.0, 75.0, 80.0,
                   96.0],
             mask=[False, False, False, False,  True, False, False, False,
                   False, False],
       fill_value=-1.0)

In [8]:
with pm.Model() as m:
    tau = pm.Gamma("tau", .001, .001)
    beta0 = pm.Normal("beta0", 0, tau=.001)
    beta1 = pm.Normal("beta1", 0, tau=.001)
    variance = pm.Deterministic("variance", 1/tau)
    
    x_imputed = pm.TruncatedNormal("x_imputed", mu=50, sigma=10, lower=0, observed=time)
    
    mu = beta0 + beta1 * x_imputed
    likelihood = pm.Normal("likelihood", mu=mu, tau=tau, observed=temp)
    
    # Bayesian R2 from fat1.odc
    sse = (n - p) * variance
    cy = temp - temp.mean()
    sst = pm.math.dot(cy, cy)
    br2 = pm.Deterministic("br2", 1 - sse/sst)
    
    trace = pm.sample(8000)
    ppc = pm.sample_posterior_predictive(trace)

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/model.py:1322: ImputationWarning: Data in x_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/model.py:1322: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, beta0, beta1, x_imputed_missing, likelihood_missing]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 8_000 draw iterations (4_000 + 32_000 draws total) took 27 seconds.
There were 57 divergences after tuning. Increase `target_accept` or reparameterize.
There were 70 divergences after

In [9]:
az.summary(trace, hdi_prob=.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,105.683,2.197,101.343,109.921,0.022,0.016,10714.0,10881.0,1.0
beta1,-0.018,0.035,-0.087,0.049,0.000,0.000,10572.0,10488.0,1.0
likelihood_missing[0],104.360,2.416,99.440,108.987,0.019,0.013,17274.0,14525.0,1.0
tau,0.282,0.153,0.032,0.579,0.002,0.002,5949.0,4064.0,1.0
x_imputed_missing[0],49.226,10.020,29.515,68.525,0.087,0.062,13079.0,10193.0,1.0
variance,5.036,4.659,1.110,11.736,0.051,0.036,5949.0,4064.0,1.0
br2,-0.401,1.297,-2.266,0.691,0.014,0.010,5949.0,4064.0,1.0


In [10]:
time2=time*time
time2

masked_array(data=[576.0, 1024.0, 2304.0, 3136.0, --, 4900.0, 5184.0,
                   5625.0, 6400.0, 9216.0],
             mask=[False, False, False, False,  True, False, False, False,
                   False, False],
       fill_value=-1.0)

In [11]:
with pm.Model() as m:
    tau = pm.Gamma("tau", .001, .001)
    beta0 = pm.Normal("beta0", 0, tau=.001)
    beta1 = pm.Normal("beta1", 0, tau=.001)
    beta2 = pm.Normal("beta2", 0, tau=.001)
    variance = pm.Deterministic("variance", 1/tau)
    
    x_imputed = pm.TruncatedNormal("x_imputed", mu=20, sigma=10, lower=0, observed=time)
    x2_imputed = pm.TruncatedNormal("x2_imputed", mu=20, sigma=10, lower=0, observed=time2)
    
    mu = beta0 + beta1 * x_imputed +beta2*x2_imputed
    likelihood = pm.Normal("likelihood", mu=mu, tau=tau, observed=temp)
    
    # Bayesian R2 from fat1.odc
    sse = (n - p) * variance
    cy = temp - sum([102.8,104.5,106.5,107.0,107.1,105.1,103.9,103.2,102.1])/9
    sst = pm.math.dot(cy, cy)
    br2 = pm.Deterministic("br2", 1 - sse/sst)
    
    trace = pm.sample(8000)
    ppc = pm.sample_posterior_predictive(trace)
    

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/model.py:1322: ImputationWarning: Data in x_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/model.py:1322: ImputationWarning: Data in x2_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/model.py:1322: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, beta0, beta1, beta2, x_imputed_missing, x2_imputed_missing, likelihood_missing]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 8_000 draw iterations (4_000 + 32_000 draws total) took 84 seconds.
There were 4519 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability do

In [42]:
az.summary(trace, hdi_prob=.95)

Got error No model on context stack. trying to find log_likelihood in translation.
/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3_3x.py:102: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
arviz - WARNING - Array contains NaN-value.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,97.673,2.955,91.662,103.385,0.116,0.082,541.0,2897.0,1.02
beta1,0.305,0.107,0.110,0.539,0.004,0.003,466.0,2309.0,1.02
beta2,-0.003,0.001,-0.005,-0.001,0.000,0.000,452.0,2618.0,1.02
x_imputed_missing[0],29.455,4.978,19.040,39.195,0.249,0.187,332.0,363.0,1.00
x2_imputed_missing[0],20.388,9.393,3.325,38.192,0.942,0.719,155.0,456.0,1.02
likelihood_missing[0],104.738,1.420,101.921,107.624,0.035,0.025,1220.0,4003.0,1.01
tau,0.920,0.511,0.117,1.939,0.106,0.085,30.0,53.0,1.07
variance,1.672,1.785,0.384,4.291,0.080,0.056,30.0,53.0,1.07
br2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#data=pd.read_csv("bladderc.csv")
data = pd.read_csv("bladderc.csv")
data["censored"] = 0
data.loc[data["observed"] == 0, "censored"] = data["time"]

y = data["time"].to_numpy(copy=True)
x = data["group"].to_numpy(copy=True)
observed = data["observed"].to_numpy(copy=True).astype(bool)
censored = data["censored"].to_numpy(copy=True)

np.sum(observed), y.shape, x.shape, observed.shape, censored.shape

x_uncensored = x[observed]
x_censored = x[~observed]
y_uncensored = y[observed]
censored = censored[~observed]

Question 2 starts here

In [191]:
data=[[0,0,0],[1,0,0],[4,0,0],[7,0,0],[10,0,0],[6,1,0],[14,0,0],[18,0,0],[5,1,0],[12,1,0],[23,0,0],[10,1,0],[3,1,0],[3,1,0],[7,1,0],[3,1,0],[26,0,0],[1,1,0],[2,1,0],[25,1,0],[29,0,0],[29,0,0],[29,0,0],[28,1,0],[2,1,0],[3,1,0],[12,1,0],[32,0,0],[34,0,0],[36,0,0],[29,1,0],[37,0,0],[9,1,0],[16,1,0],[41,0,0],[3,1,0],[6,1,0],[3,1,0],[9,1,0],[18,1,0],[49,0,0],[35,1,0],[17,1,0],[3,1,0],[59,0,0],[2,1,0],[5,1,0],[2,1,0],[1,0,1],[1,0,1],[5,1,1],[9,0,1],[10,0,1],[13,0,1],[3,1,1],[1,1,1],[18,0,1],[17,1,1],[2,1,1],[17,1,1],[22,0,1],[25,0,1],[25,0,1],[25,0,1],[6,1,1],[6,1,1],[2,1,1],[26,1,1],[38,0,1],[22,1,1],[4,1,1],[24,1,1],[41,0,1],[41,0,1],[1,1,1],[44,0,1],[2,1,1],[45,0,1],[2,1,1],[46,0,1],[49,0,1],[50,0,1],[4,1,1],[54,0,1],[38,1,1],[59,0,1]]
data=np.array(data)
#data

In [168]:
x = data[:, 2].copy()
censored_vals = data[:, 1].copy()
y = data[:, 0].copy()
#censored_vals[censored_vals == 0] = 0
censored_vals[censored_vals == 1] = y[censored_vals == 1]


# we need to separate the observed values and the censored values
observed_mask = censored_vals == 0

censored = censored_vals[~observed_mask]
y_uncensored = y[observed_mask]
x_censored = x[~observed_mask]
x_uncensored = x[observed_mask]

#censored=list(censored)
#y_uncensored=list(y_uncensored)
#x_censored=list(x_censored)
#x_uncensored=list(x_uncensored)

In [17]:
log2 = np.log(2)

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.001)
    beta1 = pm.Normal("beta1", 0, tau=0.001)
    #α = pm4.Exponential("α", 3)

    λ_censored = pm.math.exp(beta0 + beta1 * x_censored)
    #β_censored = λ_censored ** (-1 / α)

    λ_uncensored = pm.math.exp(beta0 + beta1 * x_uncensored)
    #β_uncensored = λ_uncensored ** (-1 / α)

    impute_censored = pm.Bound(
        "impute_censored",
        pm.Exponential.dist(lam=λ_censored),
        lower=censored,
        shape=λ_censored.shape[0],
    )

    likelihood = pm.Exponential(
        "likelihood",
        lam=λ_uncensored, #what is here
        observed=y_uncensored,
        shape=y_uncensored.shape,
    )

    #differ = pm.Deterministic("difference", (pm.math.exp(-beta0 - beta1))-(pm.math.exp(-beta0)))

    trace = pm.sample(
        10000, tune=2000, cores=4, init="auto", step=[pm.NUTS(target_accept=0.95)]
    )

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/pymc/sampling.py:635: UserWarning: Unable to validate shapes: Cannot sample from a bounded variable
Apply node that caused the error: bound_rv{0, (0, 0, 0), floatX, False}(RandomStateSharedVariable(<RandomState(MT19937) at 0x17846A640>), TensorConstant{(1,) of 39}, TensorConstant{11}, exponential_rv{0, (0,), floatX, False}.out, TensorConstant{[ 0.  1.  ... 54. 59.]}, TensorConstant{inf})
Toposort index: 13
Inputs types: [RandomStateType, TensorType(int64, (1,)), TensorType(int64, ()), TensorType(float64, (None,)), TensorType(float64, (39,)), TensorType(float64, ())]
Inputs shapes: ['No shapes', (1,), (), (39,), (39,), ()]
Inputs strides: ['No strides', (8,), (), (8,), (8,), ()]
Inputs values: [RandomState(MT19937) at 0x17846A640, array([39]), array(11), 'not shown', 'not shown', array(inf)]
Outputs clients: [['output'], []]

Backtrace when the node is created (use Aesara flag traceback__limit=N to make it longer):
 

/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 22 seconds.


In [66]:
x = data.iloc[:, 2].copy()
censored = data.iloc[:, 1].copy()
y = data.iloc[:, 0].copy()
# for pymc, right-censored values must be greater than or equal to than the "upper" value
#y[np.isnan(y)] = censored[np.isnan(y)]
censored[censored == 0] = np.inf
censored[censored == 1] = y[censored == 1]
x=list(x)
censored=list(censored)
y=list(y)